In [102]:
%load_ext autoreload
%autoreload 2

In [1]:
!pip install transformers

     |████████████████████████████████| 2.8 MB 4.3 MB/s eta 0:00:01     |██████████████████████▋         | 2.0 MB 4.3 MB/s eta 0:00:01
     |████████████████████████████████| 2.2 MB 33.8 MB/s eta 0:00:01
     |████████████████████████████████| 895 kB 34.4 MB/s eta 0:00:01


In [3]:
!pip install torch

     |████████████████████████████████| 127.9 MB 8.2 MB/s eta 0:00:01��████   | 116.0 MB 8.2 MB/s eta 0:00:02


In [103]:
from happysadsongs.data import *
import pandas as pd
import numpy as np
import torch
from transformers import BertTokenizer, BertModel, BertForSequenceClassification, TFPreTrainedModel, TFBertForSequenceClassification, Trainer, TrainingArguments, AdamW, get_linear_schedule_with_warmup
import string
from sklearn.preprocessing import OneHotEncoder
from tokenizers import normalizers
from tokenizers.normalizers import NFD, StripAccents
from sklearn.model_selection import train_test_split
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler

In [2]:
train_set = get_training_data()

## Basic/minimal preprocessing

### Test Modified Clean Text Function for BERT 
Keep periods

In [3]:

punctuations = set(string.punctuation) - {'!','.', ';', '?'}

In [110]:
def clean(text):
    # lowercase
    new_text = text.lower()

    # remove twitter handles
    new_text = re.sub(r"@\w+", '', new_text)

    # remove urls
    new_text = re.sub(r'http:\S+', '', new_text)
    new_text = re.sub(r'https:\S+', '', new_text)

    # remove punctuation
#     for punctuation in punctuations:
#         new_text = new_text.replace(punctuation, '')

    # remove numbers
    new_text = ''.join(word for word in new_text if not word.isdigit())

    return new_text

In [111]:
train_set['text_cleaned'] = train_set['text'].apply(clean)

In [7]:
train_set

,Unnamed: 0,text,word_label,source,text_cleaned
0,0,i am thankful that she continues to feel comfo...,happy,HuggingFace,i am thankful that she continues to feel comfo...
1,1,i want to do it the right way oh orihime whisp...,happy,HuggingFace,i want to do it the right way oh orihime whisp...
2,2,i was gaining weight getting a lot stronger an...,happy,HuggingFace,i was gaining weight getting a lot stronger an...
3,3,i feel that there is no way to determine if a ...,happy,HuggingFace,i feel that there is no way to determine if a ...
4,4,i feel sure that i wouldnt have gained so much...,happy,HuggingFace,i feel sure that i wouldnt have gained so much...
...,...,...,...,...,...
42480,1315,This hurts to hear because we know it's true. ...,sad,Google_GoEmotions,this hurts to hear because we know it's true. ...
42481,1316,Yep happening to me all the time at my college...,sad,Google_GoEmotions,yep happening to me all the time at my college...
42482,1317,"Shit, I lost the original page. Please forgive...",sad,Google_GoEmotions,"shit, i lost the original page. please forgive..."
42483,1318,I want to die,sad,Google_GoEmotions,i want to die


### Label Encoding

In [40]:
emotion_dict = {'happy': 2, 'sad': 0, 'angry': 1}
train_set['label'] = train_set.word_label.replace(emotion_dict)

In [41]:
train_set

,Unnamed: 0,text,word_label,source,text_cleaned,label
0,0,i am thankful that she continues to feel comfo...,happy,HuggingFace,i am thankful that she continues to feel comfo...,2
1,1,i want to do it the right way oh orihime whisp...,happy,HuggingFace,i want to do it the right way oh orihime whisp...,2
2,2,i was gaining weight getting a lot stronger an...,happy,HuggingFace,i was gaining weight getting a lot stronger an...,2
3,3,i feel that there is no way to determine if a ...,happy,HuggingFace,i feel that there is no way to determine if a ...,2
4,4,i feel sure that i wouldnt have gained so much...,happy,HuggingFace,i feel sure that i wouldnt have gained so much...,2
...,...,...,...,...,...,...
42480,1315,This hurts to hear because we know it's true. ...,sad,Google_GoEmotions,this hurts to hear because we know it's true. ...,0
42481,1316,Yep happening to me all the time at my college...,sad,Google_GoEmotions,yep happening to me all the time at my college...,0
42482,1317,"Shit, I lost the original page. Please forgive...",sad,Google_GoEmotions,"shit, i lost the original page. please forgive...",0
42483,1318,I want to die,sad,Google_GoEmotions,i want to die,0


### OHE Target

In [6]:
train_set

,Unnamed: 0,text,word_label,source,text_cleaned
0,0,i am thankful that she continues to feel comfo...,happy,HuggingFace,i am thankful that she continues to feel comfo...
1,1,i want to do it the right way oh orihime whisp...,happy,HuggingFace,i want to do it the right way oh orihime whisp...
2,2,i was gaining weight getting a lot stronger an...,happy,HuggingFace,i was gaining weight getting a lot stronger an...
3,3,i feel that there is no way to determine if a ...,happy,HuggingFace,i feel that there is no way to determine if a ...
4,4,i feel sure that i wouldnt have gained so much...,happy,HuggingFace,i feel sure that i wouldnt have gained so much...
...,...,...,...,...,...
42480,1315,This hurts to hear because we know it's true. ...,sad,Google_GoEmotions,this hurts to hear because we know it's true. ...
42481,1316,Yep happening to me all the time at my college...,sad,Google_GoEmotions,yep happening to me all the time at my college...
42482,1317,"Shit, I lost the original page. Please forgive...",sad,Google_GoEmotions,"shit, i lost the original page. please forgive..."
42483,1318,I want to die,sad,Google_GoEmotions,i want to die


In [40]:
encoder = OneHotEncoder(sparse=False, dtype='int')
encoder.fit(train_set[['word_label']])
feature_list = encoder.categories_[0].tolist()
train_set[feature_list]=encoder.transform(train_set[['word_label']])
train_set.drop(columns=['word_label', 'Unnamed: 0'], inplace=True)

train_set.head()

,text,source,text_cleaned,angry,happy,sad
0,i am thankful that she continues to feel comfo...,HuggingFace,i am thankful that she continues to feel comfo...,0,1,0
1,i want to do it the right way oh orihime whisp...,HuggingFace,i want to do it the right way oh orihime whisp...,0,1,0
2,i was gaining weight getting a lot stronger an...,HuggingFace,i was gaining weight getting a lot stronger an...,0,1,0
3,i feel that there is no way to determine if a ...,HuggingFace,i feel that there is no way to determine if a ...,0,1,0
4,i feel sure that i wouldnt have gained so much...,HuggingFace,i feel sure that i wouldnt have gained so much...,0,1,0


## Train Test Split

In [42]:
X = train_set['text_cleaned']
y = train_set['label']

In [43]:
X_train, X_val, y_train, y_val = train_test_split(train_set.text_cleaned.values, 
                                                  train_set.label.values, 
                                                  test_size=0.15, 
                                                  random_state=42, 
                                                  stratify=train_set.label.values)

In [44]:
y_train

array([2, 0, 1, ..., 0, 0, 1])

## BERT Encoding

In [70]:
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")

encoded_data_train = tokenizer.batch_encode_plus(
    X_train, 
    add_special_tokens=True, 
    return_attention_mask=True, 
    padding='max_length', 
    max_length=200,
    truncation=True,
    return_tensors='pt'
)

encoded_data_val = tokenizer.batch_encode_plus(
    X_val, 
    add_special_tokens=True, 
    return_attention_mask=True, 
    padding='max_length', 
    max_length=200,
    truncation=True,
    return_tensors='pt'
)


input_ids_train = encoded_data_train['input_ids']
attention_masks_train = encoded_data_train['attention_mask']
labels_train = torch.tensor(y_train)

input_ids_val = encoded_data_val['input_ids']
attention_masks_val = encoded_data_val['attention_mask']
labels_val = torch.tensor(y_val)

dataset_train = TensorDataset(input_ids_train, attention_masks_train, labels_train)
dataset_val = TensorDataset(input_ids_val, attention_masks_val, labels_val)

KeyboardInterrupt: 

### Attempt Model

In [61]:
model = BertForSequenceClassification.from_pretrained("bert-base-uncased",
                                                      num_labels=4,
                                                      output_attentions=False,
                                                      output_hidden_states=False)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

#### Data Loader 

In [66]:
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler

batch_size = 8

dataloader_train = DataLoader(dataset_train, 
                              sampler=RandomSampler(dataset_train), 
                              batch_size=batch_size)

dataloader_validation = DataLoader(dataset_val, 
                                   sampler=SequentialSampler(dataset_val), 
                                   batch_size=batch_size)

#### Optimizer and Scheduler

In [67]:
from transformers import AdamW, get_linear_schedule_with_warmup

optimizer = AdamW(model.parameters(),
                  lr=1e-5, 
                  eps=1e-8)
                  
epochs = 1

scheduler = get_linear_schedule_with_warmup(optimizer, 
                                            num_warmup_steps=0,
                                            num_training_steps=len(dataloader_train)*epochs)


#### Performance Metrics

In [68]:
from sklearn.metrics import f1_score

def f1_score_func(preds, labels):
    preds_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()
    return f1_score(labels_flat, preds_flat, average='weighted')

def accuracy_per_class(preds, labels):
    emotion_dict_inverse = {v: k for k, v in emotion_dict.items()}
    
    preds_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()

    for label in np.unique(labels_flat):
        y_preds = preds_flat[labels_flat==label]
        y_true = labels_flat[labels_flat==label]
        print(f'Class: {emotion_dict_inverse[label]}')
        print(f'Accuracy: {len(y_preds[y_preds==label])}/{len(y_true)} - {len(y_preds[y_preds==label]) / len(y_true)}\n')

### Training

In [65]:
import random
from tqdm.notebook import tqdm

seed_val = 18
random.seed(seed_val)
np.random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

def evaluate(dataloader_val):

    model.eval()
    
    loss_val_total = 0
    predictions, true_vals = [], []
    
    for batch in dataloader_val:
        
        batch = tuple(b.to(device) for b in batch)
        
        inputs = {'input_ids':      batch[0],
                  'attention_mask': batch[1],
                  'labels':         batch[2],
                 }

        with torch.no_grad():        
            outputs = model(**inputs)
            
        loss = outputs[0]
        logits = outputs[1]
        loss_val_total += loss.item()

        logits = logits.detach().cpu().numpy()
        label_ids = inputs['labels'].cpu().numpy()
        predictions.append(logits)
        true_vals.append(label_ids)
    
    loss_val_avg = loss_val_total/len(dataloader_val) 
    
    predictions = np.concatenate(predictions, axis=0)
    true_vals = np.concatenate(true_vals, axis=0)
            
    return loss_val_avg, predictions, true_vals
    
for epoch in tqdm(range(1, epochs+1)):
    
    model.train()
    
    loss_train_total = 0

    progress_bar = tqdm(dataloader_train, desc='Epoch {:1d}'.format(epoch), leave=False, disable=False)
    for batch in progress_bar:

        model.zero_grad()
        
        batch = tuple(b.to(device) for b in batch)
        
        inputs = {'input_ids':      batch[0],
                  'attention_mask': batch[1],
                  'labels':         batch[2],
                 }       

        outputs = model(**inputs)
        
        loss = outputs[0]
        loss_train_total += loss.item()
        loss.backward()

        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)

        optimizer.step()
        scheduler.step()
        
        progress_bar.set_postfix({'training_loss': '{:.3f}'.format(loss.item()/len(batch))})
         
        
    torch.save(model.state_dict(), f'bert/finetuned_BERT_epoch_{epoch}.model')
        
    tqdm.write(f'\nEpoch {epoch}')
    
    loss_train_avg = loss_train_total/len(dataloader_train)            
    tqdm.write(f'Training loss: {loss_train_avg}')
    
    val_loss, predictions, true_vals = evaluate(dataloader_validation)
    val_f1 = f1_score_func(predictions, true_vals)
    tqdm.write(f'Validation loss: {val_loss}')
    tqdm.write(f'F1 Score (Weighted): {val_f1}')

  0%|          | 0/2 [00:00<?, ?it/s]

Epoch 1:   0%|          | 0/4514 [00:00<?, ?it/s]

KeyboardInterrupt: 

### old attempt

In [49]:
def bert_tokenize(text):
    encoding = tz.encode_plus(text=text,  # the sentence to be encoded
    add_special_tokens=True,  # Add [CLS] and [SEP]
    max_length = 100,  # maximum length of a sentence
    truncation=True,
    pad_to_max_length=True,  # Add [PAD]s
    return_attention_mask = True,  # Generate the attention mask
    return_tensors = 'pt',  # ask the function to return PyTorch tensors
    )
    return encoding

In [59]:
encoded_X = X.apply(bert_tokenize)

In [88]:
train_set['encoding'] = encoded_X

In [134]:
type(encoded_X)

pandas.core.series.Series

In [139]:
input_dict = {'input_ids':[encoded_X[0]['input_ids']]}
for row in encoded_X[1:]:
    input_dict['input_ids'].append(row['input_ids'])

In [140]:
input_dict

{'input_ids': [tensor([[  101,  1045,  2572, 18836,  2008,  2016,  4247,  2000,  2514,  6625,
            3331,  2055,  2007,  2033,  1998,  3485,  2075,   102,     0,     0,
               0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
               0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
               0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
               0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
               0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
               0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
               0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
               0,     0,     0,     0,     0,     0,     0,     0,     0,     0]]),
  tensor([[  101,  1045,  2215,  2000,  2079,  2009,  1996,  2157,  2126,  2821,
            2030, 19190, 14428,  3990,  2067,  3110, 26350,  1999,  2010, 23997,
            

## Bert Model

In [143]:
model = TFBertForSequenceClassification.from_pretrained('bert-base-uncased')


All model checkpoint layers were used when initializing TFBertForSequenceClassification.

Some layers of TFBertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


## DistilBert

In [73]:
from transformers import DistilBertTokenizer, AutoTokenizer

tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased')

encoded_data_train = tokenizer.batch_encode_plus(
    X_train, 
    add_special_tokens=True, 
    return_attention_mask=True, 
    padding='max_length', 
    max_length=200,
    truncation=True,
    return_tensors='pt'
)

encoded_data_val = tokenizer.batch_encode_plus(
    X_val, 
    add_special_tokens=True, 
    return_attention_mask=True, 
    padding='max_length', 
    max_length=200,
    truncation=True,
    return_tensors='pt'
)


input_ids_train = encoded_data_train['input_ids']
attention_masks_train = encoded_data_train['attention_mask']
labels_train = torch.tensor(y_train)

input_ids_val = encoded_data_val['input_ids']
attention_masks_val = encoded_data_val['attention_mask']
labels_val = torch.tensor(y_val)

dataset_train = TensorDataset(input_ids_train, attention_masks_train, labels_train)
dataset_val = TensorDataset(input_ids_val, attention_masks_val, labels_val)

In [82]:
from transformers import DistilBertForSequenceClassification, Trainer, TrainingArguments

args = TrainingArguments(
    output_dir="bert",
    evaluation_strategy = "steps",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=1,
    weight_decay=0.01,
    load_best_model_at_end=True,
    metric_for_best_model="accuracy",
)

model = DistilBertForSequenceClassification.from_pretrained("distilbert-base-uncased")

trainer = Trainer(
    model=model,                         # the instantiated 🤗 Transformers model to be trained
    args=args,                  # training arguments, defined above
    train_dataset=dataset_train,         # training dataset
    eval_dataset=dataset_val             # evaluation dataset
)



using `logging_steps` to initialize `eval_steps` to 500
PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).
loading configuration file https://huggingface.co/distilbert-base-uncased/resolve/main/config.json from cache at /Users/vprentice/.cache/huggingface/transformers/23454919702d26495337f3da04d1655c7ee010d5ec9d77bdb9e399e00302c0a1.91b885ab15d631bf9cee9dc9d25ece0afd932f2f5130eba28f2055b2220c0333
Model config DistilBertConfig {
  "activation": "gelu",
  "architectures": [
    "DistilBertForMaskedLM"
  ],
  "attention_dropout": 0.1,
  "dim": 768,
  "dropout": 0.1,
  "hidden_dim": 3072,
  "initializer_range": 0.02,
  "max_position_embeddings": 512,
  "model_type": "distilbert",
  "n_heads": 12,
  "n_layers": 6,
  "pad_token_id": 0,
  "qa_dropout": 

In [85]:
import mlflow 
mlflow.end_run()

In [86]:
trainer.train()

***** Running training *****
  Num examples = 36112
  Num Epochs = 1
  Instantaneous batch size per device = 16
  Total train batch size (w. parallel, distributed & accumulation) = 16
  Gradient Accumulation steps = 1
  Total optimization steps = 2257


TypeError: vars() argument must have __dict__ attribute

In [48]:
# Magda's model

def init_model():
    model = Sequential()
    model.add(layers.Masking())
    model.add(layers.LSTM(20, activation='tanh'))
    model.add(layers.Dense(15, activation='relu'))
    model.add(layers.Dense(3, activation='softmax'))

    model.compile(loss='categorical_crossentropy',
                  optimizer='rmsprop',
                  metrics=[['accuracy','Precision','Recall']])
    
    return model

model = init_model()

NameError: name 'Sequential' is not defined

In [87]:
model = DistilBertForSequenceClassification.from_pretrained("distilbert-base-uncased",
                                                      num_labels=4,
                                                      output_attentions=False,
                                                      output_hidden_states=False)

loading configuration file https://huggingface.co/distilbert-base-uncased/resolve/main/config.json from cache at /Users/vprentice/.cache/huggingface/transformers/23454919702d26495337f3da04d1655c7ee010d5ec9d77bdb9e399e00302c0a1.91b885ab15d631bf9cee9dc9d25ece0afd932f2f5130eba28f2055b2220c0333
Model config DistilBertConfig {
  "activation": "gelu",
  "architectures": [
    "DistilBertForMaskedLM"
  ],
  "attention_dropout": 0.1,
  "dim": 768,
  "dropout": 0.1,
  "hidden_dim": 3072,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1",
    "2": "LABEL_2",
    "3": "LABEL_3"
  },
  "initializer_range": 0.02,
  "label2id": {
    "LABEL_0": 0,
    "LABEL_1": 1,
    "LABEL_2": 2,
    "LABEL_3": 3
  },
  "max_position_embeddings": 512,
  "model_type": "distilbert",
  "n_heads": 12,
  "n_layers": 6,
  "pad_token_id": 0,
  "qa_dropout": 0.1,
  "seq_classif_dropout": 0.2,
  "sinusoidal_pos_embds": false,
  "tie_weights_": true,
  "transformers_version": "4.10.0",
  "vocab_size": 30522
}

loading

#### Data Loader 

In [88]:
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler

batch_size = 8

dataloader_train = DataLoader(dataset_train, 
                              sampler=RandomSampler(dataset_train), 
                              batch_size=batch_size)

dataloader_validation = DataLoader(dataset_val, 
                                   sampler=SequentialSampler(dataset_val), 
                                   batch_size=batch_size)

#### Optimizer and Scheduler

In [89]:
from transformers import AdamW, get_linear_schedule_with_warmup

optimizer = AdamW(model.parameters(),
                  lr=1e-5, 
                  eps=1e-8)
                  
epochs = 1

scheduler = get_linear_schedule_with_warmup(optimizer, 
                                            num_warmup_steps=0,
                                            num_training_steps=len(dataloader_train)*epochs)


#### Performance Metrics

In [90]:
from sklearn.metrics import f1_score

def f1_score_func(preds, labels):
    preds_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()
    return f1_score(labels_flat, preds_flat, average='weighted')

def accuracy_per_class(preds, labels):
    emotion_dict_inverse = {v: k for k, v in emotion_dict.items()}
    
    preds_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()

    for label in np.unique(labels_flat):
        y_preds = preds_flat[labels_flat==label]
        y_true = labels_flat[labels_flat==label]
        print(f'Class: {emotion_dict_inverse[label]}')
        print(f'Accuracy: {len(y_preds[y_preds==label])}/{len(y_true)} - {len(y_preds[y_preds==label]) / len(y_true)}\n')

### Training

In [91]:
import random
from tqdm.notebook import tqdm

seed_val = 18
random.seed(seed_val)
np.random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

def evaluate(dataloader_val):

    model.eval()
    
    loss_val_total = 0
    predictions, true_vals = [], []
    
    for batch in dataloader_val:
        
        batch = tuple(b.to(device) for b in batch)
        
        inputs = {'input_ids':      batch[0],
                  'attention_mask': batch[1],
                  'labels':         batch[2],
                 }

        with torch.no_grad():        
            outputs = model(**inputs)
            
        loss = outputs[0]
        logits = outputs[1]
        loss_val_total += loss.item()

        logits = logits.detach().cpu().numpy()
        label_ids = inputs['labels'].cpu().numpy()
        predictions.append(logits)
        true_vals.append(label_ids)
    
    loss_val_avg = loss_val_total/len(dataloader_val) 
    
    predictions = np.concatenate(predictions, axis=0)
    true_vals = np.concatenate(true_vals, axis=0)
            
    return loss_val_avg, predictions, true_vals
    
for epoch in tqdm(range(1, epochs+1)):
    
    model.train()
    
    loss_train_total = 0

    progress_bar = tqdm(dataloader_train, desc='Epoch {:1d}'.format(epoch), leave=False, disable=False)
    for batch in progress_bar:

        model.zero_grad()
        
        batch = tuple(b.to(device) for b in batch)
        
        inputs = {'input_ids':      batch[0],
                  'attention_mask': batch[1],
                  'labels':         batch[2],
                 }       

        outputs = model(**inputs)
        
        loss = outputs[0]
        loss_train_total += loss.item()
        loss.backward()

        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)

        optimizer.step()
        scheduler.step()
        
        progress_bar.set_postfix({'training_loss': '{:.3f}'.format(loss.item()/len(batch))})
         
        
    torch.save(model.state_dict(), f'bert/finetuned_distilBERT_epoch_{epoch}.model')
        
    tqdm.write(f'\nEpoch {epoch}')
    
    loss_train_avg = loss_train_total/len(dataloader_train)            
    tqdm.write(f'Training loss: {loss_train_avg}')
    
    val_loss, predictions, true_vals = evaluate(dataloader_validation)
    val_f1 = f1_score_func(predictions, true_vals)
    tqdm.write(f'Validation loss: {val_loss}')
    tqdm.write(f'F1 Score (Weighted): {val_f1}')

  0%|          | 0/1 [00:00<?, ?it/s]

Epoch 1:   0%|          | 0/4514 [00:00<?, ?it/s]


Epoch 1
Training loss: 0.5774658351210654
Validation loss: 0.5015045166006277
F1 Score (Weighted): 0.8024620183575282


In [92]:
_, predictions, true_vals = evaluate(dataloader_validation)
accuracy_per_class(predictions, true_vals)

Class: sad
Accuracy: 1555/2100 - 0.7404761904761905

Class: angry
Accuracy: 1672/2094 - 0.7984718242597899

Class: happy
Accuracy: 1890/2179 - 0.867370353373107



### Test Lyrics

In [107]:
dataset_lyrics = get_test_lyrics()

In [112]:
dataset_lyrics['cleaned_lyrics'] = dataset_lyrics.lyrics.apply(clean)

lyrics_target = dataset_lyrics['label'].map(emotion_dict)

encoded_data_lyr = tokenizer.batch_encode_plus(
    dataset_lyrics['cleaned_lyrics'], 
    add_special_tokens=True, 
    return_attention_mask=True, 
    padding='max_length', 
    max_length=200,
    truncation=True,
    return_tensors='pt'
)


input_ids_lyr = encoded_data_lyr['input_ids']
attention_masks_lyr = encoded_data_lyr['attention_mask']
labels_lyr = torch.tensor(lyrics_target)


In [113]:
dataset_lyr = TensorDataset(input_ids_lyr, attention_masks_lyr, labels_lyr)

In [114]:
dataloader_lyrics = DataLoader(dataset_lyr, 
                                   sampler=SequentialSampler(dataset_lyr), 
                                   batch_size=batch_size)

In [115]:
val_loss, predictions, true_vals = evaluate(dataloader_lyrics)
val_f1 = f1_score_func(predictions, true_vals)

In [116]:
val_f1

0.6636297728470626

In [117]:
accuracy_per_class(predictions, true_vals)

Class: sad
Accuracy: 62/100 - 0.62

Class: angry
Accuracy: 32/55 - 0.5818181818181818

Class: happy
Accuracy: 79/105 - 0.7523809523809524



In [119]:
true_vals

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1])

In [120]:
predictions

array([[ 1.4681511 ,  0.05535065,  1.9693245 , -4.29678   ],
       [ 1.8004179 , -0.12136885,  1.9361078 , -4.174942  ],
       [ 4.047917  , -0.27958143, -0.0735831 , -3.0801492 ],
       ...,
       [ 2.4078176 ,  1.734954  , -0.32155013, -3.9325604 ],
       [ 1.3306773 ,  1.4241192 ,  0.9623521 , -4.3915906 ],
       [ 1.0268732 ,  2.3207264 ,  0.34947455, -4.37497   ]],
      dtype=float32)